In [1]:
from functools import wraps, partial

def debug(func=None, *, prefix='***'):
    if func is None:
        return partial(debug, prefix=prefix)
    
    msg = prefix + func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(msg)
        return func(*arg, **kwargs)
    return wrapper

def debugmethod(cls):
    for name, val in vars(cls).items():
        if callable(val):
            setattr(cls, name, debug(val))
    return cls

class debugmeta(type):
    def __new__(cls, clsname, bases, clsdict):
        clsobj = super().__new__(cls, clsname, bases, clsdict)
        clsobj = debugmethods(clsobj)
        return clsobj

## What happens during the class definition
```
Class Spam(Base):  
    def __init__(self, i):  
        pass  
    def some_method(self):  
        pass  
```
1. Body of class is isolated
2. The class dictionary is created:  
    1. clsdict = type.\__prepare\__('Spam', (Base,))  
    2. This dictionary serves as local namespace for statements in the class body.  
    3. By default, its a simple dictionary.
3. Body is executed in returned dict.  
    1. exec(body, globals(), clsdict)
    2. Afterwards, clsdict is populated.
4. Class is constructed from its name, base classes and the dictionary
```
Spam = type('Spam', (Base,), clsdict)
Spam
<class '__main__.Spam'>
```

## Changing the Metaclass
```
class Spam(metaclass=type):
    pass
```
By default it's set to type, but you can change it to sth else. (replacing the step 4)

### You typically inherit from type and redefine \__new\__ or \__init\__

In [2]:
class mytype(type):
    def __new__(cls, clsname, bases, clsdict):
        print('Boom! Creating a class named: {}'.format(clsname))
        return super().__new__(cls, clsname, bases, clsdict)
    
class spam(metaclass=mytype):
    pass

Boom! Creating a class named: spam


## Why use metaclass?
## Metaclasses **propagate** down hierarchies

In [3]:
class Stock:
    def __init__(self, name, shares, price):
        self.__dict__.update({k: v for k, v in locals().items() if k != 'self'})

class Point:
    def __init__(self, x, y):
        self.__dict__.update({k: v for k, v in locals().items() if k != 'self'})
        
class Address:
    def __init__(self, hostname, port):
        self.__dict__.update({k: v for k, v in locals().items() if k != 'self'})

In [4]:
# A general version
class Structure:
    _fields = []
    def __init__(self, *args):
        for name, val in zip(self._fields, args):
            setattr(self, name, val)
            
class Stock(Structure):
    _fields = ['name', 'shares', 'price']
    
class Pointer(Structure):
    _fields = ['x', 'y']
    
class Addres(Structure):
    _fields = ['hostname', 'port']

## The generalization have some downsides as well, the help() functino won't work and you losses keyword arguments! We fix it with `signature`!

In [5]:
from inspect import Parameter, Signature

def make_signature(names):
    return Signature(
    Parameter(name, Parameter.POSITIONAL_OR_KEYWORD) for name in names)

class Structure:
    __signature__ = make_signature([])
    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(*args, **kwargs)
        for name, val in bound.arguments.items():
            setattr(self, name, val)
            
class Stock(Structure):
    __signature__ = make_signature(['name', 'shares', 'price'])

### A even better way! Use metaclasses

In [6]:
class StructMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        clsobj = super().__new__(cls, clsname, bases, clsdict)
        sig = make_signature(clsobj._fields)
        setattr(clsobj, '__signature__', sig)
        return clsobj

class Structure(metaclass=StructMeta):
    _fields = []
    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(*args, **kwargs)
        for name, val in bound.arguments.items():
            setattr(self, name, val)
            
class Stock(Structure):
    _fields = ['name', 'shares', 'price']
    

## Owning the dots


In [7]:
class Descriptor:
    def __init__(self, name=None):
        self.name = name
        
    def __get__(self, instance, cls):
        print("Get", self.name)
        return instance.__dict__[self.name]
        
    def __set__(self, instance, value):
        #type checker
        print('Set', self.name, value)
        instance.__dict__[self.name] = value
         
    def __delete(self, instance):
        print("Delete", self.name)
        del instance.__dict__[self.name]


In [15]:
#Type checking 
class Typed(Descriptor):
    ty = object
    def __set__(self, instance, value):
        if not isinstance(value, self.ty):
            raise TypeError('Excepted %s' % self.ty)
        super().__set__(instance, value)
        

class Integer(Typed):
    ty = int
    
class Float(Typed):
    ty = float
    
class String(Typed):
    ty = str
    
class Stock(Structure):
    _fields = ['name', 'shares', 'price']
    name = String('name')
    shares = Integer('shares')
    price = Float('price')
    
s = Stock('Some', 2, 3.5)
s.name = 5 # check typing

Set name Some
Set shares 2
Set price 3.5


TypeError: Excepted <class 'str'>

In [29]:
class Positive(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Must be >= 0')
        super().__set__(instance, value)
        

class PositiveInteger(Integer, Positive):
    # The order of mixins matters!
    pass

class PositiveFloat(Float, Positive):
    pass

class Stock(Structure):
    _fields = ['name', 'shares', 'price']
    name = String('name')
    shares = PositiveInteger('shares')
    price = PositiveFloat('price')


s = Stock('apple', 1, 2.2)
s.shares = -5

Set name apple
Set shares 1
Set price 2.2


ValueError: Must be >= 0

In [32]:
PositiveInteger.__mro__ # The order in which the __set__ is checked

(__main__.PositiveInteger,
 __main__.Integer,
 __main__.Typed,
 __main__.Positive,
 __main__.Descriptor,
 object)

In [42]:
#length checking
class Sized(Descriptor):
    def __init__(self, *args, maxlen, **kwargs):
        # maxlen is a keyword only argument
        self.maxlen = maxlen
        super().__init__(*args, **kwargs)
        
    def __set__(self, instance, value):
        if len(value) > self.maxlen:
            raise ValueError('Too big')
        super().__set__(instance, value)
        

class SizedString(String, Sized):
    pass

class Stock(Structure):
    _fields = ['name', 'shares', 'price']
    name = SizedString('name', maxlen=5)
    shares = PositiveInteger('shares')
    price = PositiveFloat('price')

    
s = Stock('apple', 5, 12.1)
s.name = 'toolongforname'

Set name apple
Set shares 5
Set price 12.1


ValueError: Too big

## Let's push it further

In [43]:
from collections import OrderedDict

In [ ]:
class StructMeta(type):
    @classmethod
    def __prepare__(cls, name, bases):
        #the second step in creating a class
        return OrderedDict()
    
    def __new__(cls, clsname, bases, clsdict):
        fileds = [key for key, val in clsdict.items()
                 if isinstance(val, Descriptor)]
        
        for name in fileds:
            clsdict[name].name = name
            
        clsobj = super().__new__(cls, clsname, bases, dict(clsdict))
        # !!notice here you need to convert ordereddict to dict
        # because python only takes dictionary when creating a class
        # the rest of the code is the sameI

In [ ]:
class NoDupOrderedDict(OrderedDict):
    def __setitem__(self, key, value):
        if key in self:
            raise NameError('Duplicates boy')
        super().__setitem__(key, value)


In [45]:
import sys
sys.meta_path

[_frozen_importlib.BuiltinImporter,
 _frozen_importlib.FrozenImporter,
 _frozen_importlib_external.PathFinder,

In [56]:
#change the way packages are imported
class MyFinder:
    def find_module(self, fullname, path):
        print(fullname, path)
        return None
    
sys.meta_path.insert(0, MyFinder())

In [72]:
import math
print(dir(math))
math.__file__

['__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'acos', 'acosh', 'asin', 'asinh', 'atan', 'atan2', 'atanh', 'ceil', 'copysign', 'cos', 'cosh', 'degrees', 'e', 'erf', 'erfc', 'exp', 'expm1', 'fabs', 'factorial', 'floor', 'fmod', 'frexp', 'fsum', 'gamma', 'gcd', 'hypot', 'inf', 'isclose', 'isfinite', 'isinf', 'isnan', 'ldexp', 'lgamma', 'log', 'log10', 'log1p', 'log2', 'modf', 'nan', 'pi', 'pow', 'radians', 'sin', 'sinh', 'sqrt', 'tan', 'tanh', 'tau', 'trunc']


'/home/maxliu/anaconda3/lib/python3.6/lib-dynload/math.cpython-36m-x86_64-linux-gnu.so'

In [83]:
import inspect
import math

In [159]:
# annotations
from inspect import signature
from functools import partial, wraps


def typeassert(*type_args):
    

    def decorator(func):
        if func is None:
            return partial(typeassert, *type_args)
        
        params = signature(func).parameters
        if len(type_args) != len(params):
            raise ValueError("The number of types doesn't match with the number of arguments.")
        
        for arg, t in zip(params.keys(), type_args):
            assert params[arg].annotation == t, "Wrong type: expected {} got {}".format(t, params[arg].annotation)
        
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs)
        return wrapper
 
    return decorator


def typeassert2(func=None, *, type_args):
    #here type_args is a tuple
    
    if func is None:
        return partial(typeassert, type_args=type_args)
    
    params = signature(func).parameters
    if len(type_args) != len(params):
        raise ValueError("The number of types doesn't match with the number of arguments.")
    
    for arg, t in zip(params.keys(), type_args):
        assert params[arg].annotation == t, "Wrong type: expected {} got {}".format(t, params[arg].annotation)
    
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper


def cls_typeassert(*type_args):

    def decorator(cls):
        for name, val in vars(cls).items():
            if callable(val):
                setattr(cls, name, typeassert(*type_args)(val))
        return cls

    return decorator


@cls_typeassert(int, int)
class A:

    def a(x: int, y:int):
        pass

    def b(x: int, y: int):
        pass

In [165]:
import ast

code = """
for i in range(10):
    print(i)
"""
c = ast.parse(code)
ast.dump(c)

"Module(body=[For(target=Name(id='i', ctx=Store()), iter=Call(func=Name(id='range', ctx=Load()), args=[Num(n=10)], keywords=[]), body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Name(id='i', ctx=Load())], keywords=[]))], orelse=[])])"

In [ ]:
ZZ9946FVPK43S896